In [1]:
import numpy as np
from gboosting import XGBTreeModel

In [2]:
X = np.random.random_sample(size=(100, 10))
y = np.random.random_sample(size=(100,))

In [4]:
my_gbm = XGBTreeModel(n_estimators=10)
my_gbm.fit(X, y, verbose=True)

100%|██████████| 10/10 [00:00<00:00, 23032.97it/s]


In [5]:
my_gbm.predict(X)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])